# Forecast Exploration Notebook
This notebooks on predicting future performance of the world economy and financial markets based on financial time sereis and the world events time series (generated by the NLP workstream).

## DeepAR

In [3]:
# pip install "gluonts[torch]"

In [16]:
# Importing the necessary packages 
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.torch import DeepAREstimator

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [44]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/AileenNielsen/"
    "TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv",
    index_col=0,
    parse_dates=True,
)
dataset = PandasDataset(df, target="#Passengers")
df.head()

,#Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


In [61]:
# Load data from a CSV file into a PandasDataset
file_path = "../../data_engineering/timeseries_data/ts_data_lda_model_topics_10_passes_10_cnbc_news_dataset_processed.csv"
df = pd.read_csv(
    file_path,
    # parse_dates=True,
).rename(columns={'Unnamed: 0': 'Day'})
df['Day'] = df['Day'].apply(lambda x: datetime.fromisoformat(x))

df = df.set_index('Day')
df = df[['0']]
df.head()
dataset = PandasDataset(df, target="1", freq='D')

In [57]:
# Split the data for training and testing
training_data, test_gen = split(dataset, offset=-64)
test_data = test_gen.generate_instances(prediction_length=32, windows=2)

In [58]:
# Train the model and make predictions
model = DeepAREstimator(
    prediction_length=32, freq="D", trainer_kwargs={"max_epochs": 5}
).train(training_data)

forecasts = list(model.predict(test_data.input))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name  | Type        | Params | In sizes | Out sizes   
----------------------------------------------------------------
0 | model | DeepARModel | 25.9 K | ?        | [1, 100, 32]
----------------------------------------------------------------
25.9 K    Trainable params
0         Non-trainable params
25.9 K    Total params
0.104     Total estimated model params size (MB)


Epoch 0: : 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/gluonts/dataset/pandas.py:174: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df = df.to_period(freq=self.freq)


AssertionError: Dataframe index is not uniformly spaced. If your dataframe contains data from multiple series in the same column ("long" format), consider constructing the dataset with `PandasDataset.from_long_dataframe` instead.

In [ ]:
# Plot predictions
plt.plot(df["1954":], color="black")
for forecast in forecasts:
  forecast.plot()
plt.legend(["True values"], loc="upper left", fontsize="xx-large")